# Deutsche's Algorithm

*By Chayapol "Due" Hongsrimuang, G00388741*

---

## Introduction
Deutsche's algorithm is a quantum query algorithm. It is an improved version of David Deutsch's 1985 paper, where it serves as a Theoretical foundation of quantum computing.

This algorithm makes use of Hadamard gates and a query gate to compute the result. However, Pauli operations may be involved as well, depending on the state that is passed into the circuit.

While the algorithm is simple to understand, the basics of quantum computing would be needed to understand the algorithm itself. This notebook will walkthrough the concepts covered in the algorithm before tackling on the algorithm itself.

# Quantum information
Quantum information serves as a way to describe the quantum state of a computing system. A quantum system has a set of two quantum states, those being quantum bits (or qubits, for short). The state is represented by a column vector with the system described as follows:
* All the entires are complex numbers
* The sum of the **absolute values squared** of the entries must be equal to 1.
This differs from classical information, where it only concerns non-negative numbers and adding all the entries to 1 without any prerequisite calculations.

The system's column vector is described by this formula.
$$
||v|| = \sqrt{\sum_{k=1}^n |\alpha_k|^2}
$$

Qubit states are not only described by just 0 and 1, but also with + and - states, which are used in Hadamard gates described later on.

Dirac notations (or bra-ket notation) can be used to describe the qubit state vectors, as this form:
$$
|0\rangle \; |1\rangle
$$
When expanded, it produces these column vectors:
$$
|0\rangle = \left( \begin{array}{c} 1 \\ 0 \end{array} \right), |1\rangle = \left( \begin{array}{c} 0 \\ 1 \end{array} \right)
$$
The number 1 is placed where the location of each state vector is, e.g $ |0\rangle $ has 1 in the top position.

In Python, this can be in a form of an array for each state vector:

In [5]:
import numpy as np

qv0 = np.array([1, 0])
qv1 = np.array([0, 1])

## Deterministic Operations

Operations can be done on these states, and with two entries of the system: $ \Sigma = \{ 0, 1 \} $, deterministic operations can be applied onto to produce their functions. These are completely determined by input, without any probability as a factor.

With this, a form of the states can be derived to create a function.

$$ f: \Sigma \rightarrow \Sigma $$

In the case of the two entries mentioned, there are four possible functions that can be derived from via the form above.

![Picture of values when deterministic operations are done on the entries](pictures/quantum_function_mapping.png)

*Figure 1. Values when deterministic operations are done on the entries - from IBM Quantum Learning*

* The functions of $ f_1 $ and $ f_4 $ are considered as **constant** functions, where they produce constant outputs of 0 and 1 respectively.
* The functions of $ f_2 $ and $ f_3 $ are considered as **balanced** functions, where two values appear equally for each possible outcomes.
    - $ f_2 $ is an identity function, where an input is equal to the output
    - $ f_3 $ is a NOT function, or a bit flip

These functions would play a key role in Deutsch's algorithm described later on.

# Operations (or Gates)
Deutsche's algorithm involves two key unitary operations to be applied for each bit passing through. Those being Pauli operations and Hadamard operations.

## Pauli Operations

Pauli operations contain a set of 3 matrices, each representing flips.

$$
\sigma_x =
\begin{pmatrix}
0 & 1 \\
1 & 0 
\end{pmatrix} \;
\sigma_y =
\begin{pmatrix}
0 & -i \\
i & 0 
\end{pmatrix} \;
\sigma_z =
\begin{pmatrix}
1 & 0 \\
0 & -1 
\end{pmatrix}
$$
where $ i $ is an imaginary number equalling to $ \sqrt{-1} $

These operators have different purposes affecting the bit that is input in.
* $ \sigma_x $ = bit flip operator
* $ \sigma_y $ = bit and phase flip operator
* $ \sigma_z $ = phase flip operator

These are denoted via these Operator in Python. Each operator takes in a list of matrix rows.

In [6]:
from qiskit.quantum_info import Operator

# j denotes the imaginary number
X = Operator([[0, 1], [1,0]])
Y = Operator([[0, -1.0j], [1.0j, 0]])
Z = Operator([[1,0], [0, -1]])

## Hadamard Operation

Hadamard operation (or Hadamard gate) is a unitary operation on a qubit, represented by H, and has these basic qubit states:

$$
|0\rangle = \frac{|0\rangle+|1\rangle}{\sqrt{2}},\:
|1\rangle = \frac{|0\rangle-|1\rangle}{\sqrt{2}}
$$

Which maps into the following matrix.

$$
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
$$

Each qubit state, when parsed through would transform the qubit between the regular 0 and 1 state with the + and - state as follow:

$$
H|0\rangle = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
\begin{pmatrix} 1 \\ 0 \end{pmatrix} = 
\begin{pmatrix}
\frac{1}{\sqrt{2}}\\
\frac{1}{\sqrt{2}}
\end{pmatrix} =
|+\rangle
$$
$$
H|1\rangle = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
\begin{pmatrix} 0 \\ 1 \end{pmatrix} = 
\begin{pmatrix}
\frac{1}{\sqrt{2}}\\
- \frac{1}{\sqrt{2}}
\end{pmatrix} =
|-\rangle
$$

The $ |+\rangle $ state can be expanded to a linearity approach.
$$
|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)
$$

This operation can be denotated with this Operator.

In [7]:
H = Operator([[1 / np.sqrt(2), 1 / np.sqrt(2)], [1 / np.sqrt(2), -1 / np.sqrt(2)]])

# Querying
Querying the main key part of quantum computing, In opposed to the classical way of directly getting an output from a computation via an input, the output is produced by making multiple queries to the computation

![Picture of quantum querying model](pictures/quantum_query.png)

*Figure 2. Quantum querying model - from IBM Quantum Learning*

The computation that is providing the output, is often called a **black box**, as we often don't know how the computation works.

This, therefore, can be represented by this mapping:

$$
f : \Sigma^n \rightarrow \Sigma^m
$$

The letters $n$ and $m$ represents the legnth of each string. The function maps the string of length n to length m, when string is either 0 or a 1.

This query model solves some of the natural query problems that can occur. One particular problem that Deutsch's Algorithm is the parity problem.

## Parity Problem
The parity can be treated as an XOR of the bits given. The input and output are described as follows:
* Input: form of $ f : \Sigma^n \rightarrow \Sigma^m $
* Output: if $f(x) = 1 $ for number of strings $ x \in \Sigma^n $
    - 0: an even number
    - 1: an odd number

The function can be represented by a sequence of $ 2^n $ bits. The key problem to this issue is to compute the XOR in the sequence.

## Query gates
Query gates allow queries to be made. In quantum circuit models, they are **unitary** (compared to Boolean circuit models, where they compute the input function directly).

This can be identified as this equation:
$$
U_f(|y\rangle|x\rangle) = |y\oplus f(x)\rangle|x\rangle
$$
This query gate acts on $ n + m $ qubits, for all $ x \in \Sigma^n $ and $ y \in \Sigma^m $. The output for $ |y\oplus f(x)\rangle $ is done by XOR-ing them together.

This gate is a deterministic operation, a permutation matrix would be produced from this. A permutation matrix can be described to have only a 1 in each row, and only a 1 in each column, with the rest being 0s. Example being this matrix:

$$
\begin{bmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
$$

## Phase Kickback
This applies with this formula:
$$
| b \oplus c \rangle = X^c|b\rangle
$$
* If a bit is XOR-ed by a value of 1, it turns into a NOT gate.
* If a bit is XOR-ed by a value of 0, it will not do anything, but instead it is an identity function.

This phase kickback phenomenon is done in the case of when we use query gates as follows:
$$ 
U_f(|b\rangle|a\rangle) = |b \oplus f(a)\rangle|a\rangle = (X^{f(a)}|b\rangle)a\rangle
$$
And that the X operation applying to the minus state gives -1 times the minus state as follows:
$$
X|-\rangle = -|-\rangle
$$
With this in mind, the state vector can be simplified by the following:
$$
U_f(|-\rangle|a\rangle) = (-1)^{f(a)}|-\rangle|a\rangle
$$
This phase kickback phenomenon will become a key part of query gates in Deutsch's algorithm.

# The Algorithm
With the basics covered, the algorithm is described as follows in Qiskit:

In [8]:
import qiskit as qc

# From IBM Quantum Learning
def deutsch(case: int):
    if case not in [1, 2, 3, 4]:
        raise ValueError("Number passed must be in 1-4")

    f = qc.QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f

def compile(function: qc.QuantumCircuit):
    n = function.num_qubits - 1
    c = qc.QuantumCircuit(n + 1, n)

    c.x(n)
    c.h(range(n+1))

    c.barrier()
    c.compose(function, inplace=True)
    c.barrier()

    c.h(range(n))
    c.measure(range(n), range(n))

    return c

compile(deutsch(3)).draw()


┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

With this algorithm, this can be separated into 5 parts using $ \psi $
* $ |\psi_0\rangle $ (beginning of the circuit)
    - The initial state started with $ |00\rangle $
* $ |\psi_1\rangle $ = the bottom qubit is passed through the X gate, a Pauli operation. 
    - The state is now $ |01\rangle $
* $ |\psi_2\rangle $ = the qubits are passed through the Hadamard gates, changing the state from binary to signs (+ or -)
    - The state is now $ |+-\rangle $ or written as
    $$
        |\psi_2\rangle = \frac{1}{\sqrt{2}}(|0\rangle|-\rangle + |1\rangle|-\rangle)
    $$
* $ |\psi_3\rangle $ = the state is now passed into the query gate, with phase kickback occuring during this
    $$
        |\psi_3\rangle = U_f\frac{1}{\sqrt{2}}(|0\rangle|-\rangle + |1\rangle|-\rangle)
    $$
    $$
        |\psi_3\rangle = \frac{1}{\sqrt{2}}((-1)^{f(0)}|0\rangle + (-1)^{f(1)}|1\rangle)
    $$
    - This can then be deduced, depending on the value of the XOR of the two functions, f(0) and f(1)
    $$
        |\psi_3\rangle = 
        \left\{\begin{array}{rcl}
        (-1)^{f(0)}|-\rangle|+\rangle & f(0) \oplus f(1) = 0 \\
        (-1)^{f(0)}|-\rangle|-\rangle & f(0) \oplus f(1) = 1
        \end{array}\right.

    $$
* $ |\psi_4\rangle $ = the top qubit is passed through the Hadamard gate, giving a certainty to the problem.
    $$
        |\psi_4\rangle = 
        \left\{\begin{array}{rcl}
        (-1)^{f(0)}|-\rangle|0\rangle & f(0) \oplus f(1) = 0 \\
        (-1)^{f(0)}|-\rangle|1\rangle & f(0) \oplus f(1) = 1
        \end{array}\right.
    $$
    $$
        |\psi_4\rangle = (-1)^{f(0)}|-\rangle|f(0) \oplus f(1)\rangle
    $$
    - The top qubit is then measured, with XOR of f(0) and f(1)
    
Interference makes this algorithm work by computing f(0) and f(1) at the same time, with the phase kickback allowing constructive interference for the correct answer or destructive interference for the wrong answer.

(insert more analysis from ian's notes and parity problem here)

# Resources Used
* IBM Quantum Learning (Basics of quantum information - Single systems) = https://learning.quantum.ibm.com/course/basics-of-quantum-information/single-systems
* IBM Quantum Learning (Fundamentals of Quantum algorithms - Quantum query algorithms) = https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms

---
End